In [83]:
def printMaxMemUsage():
    import resource
    res = float(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)/(1024**2)
    print("Maximum memory usage during this session was: {} MegaBytes.".format(res))
    return

def printResults(E, P, Q, res, orderP, m, fbs, attempts, time):
    print("\nResults: \nGroup: {},\n#P = {},\nP = {},\nQ = {},\nres = {},\nm = {},\nFactor base size = {},\n\
Number of attempts = {},\nTotal time = {} seconds,\nResult is valid = {}."\
      .format(E,orderP,P,Q,res,m,fbs,attempts, time, res*P == Q))    
#https://www.cs.bham.ac.uk/~petitcz/files/IWSEC2013.pdf
#general alg explained well

def construct3rdSumPoly(E):
    """
    Constructs the 3rd (Semaev) summation polynomial
    for the input elliptic curve in the short Weierstrass form.

    INPUT:

    - ``E``   - an elliptic curve in the short Weierstrass form.

    Elliptic curve E is defined by the short Weierstrass
    equation: 
    Y^2 = X^3 + A*X + B
    
    In Sage this elliptic curve can be constructed this way 
    (using the long Weierstrass equation):

    E = EllipticCurve(FF, [0,0,0,A,B])

    Coefficients ``A`` and ``B`` must be elements of the finite field ``FF``.
    
    OUTPUT: Returns the 3rd summation polynomial for the elliptic
    curve ``E``. If ``E`` is not in the short Weierstrass form,
    this function raises a TypeError exception.
    
    The summation polynomial ``smp`` in variables 
    ``x1``, ``x2``, ``x3`` over a polynomial ring FF[x1,x2,x3]. 
    ``smp(a,b,c) = 0`` if and only if there are three points on the elliptic
    curve E(A,B,FF) such that:
    
    ``+-(a,y_0) +- (b,y_1) +- (c,y_2) = E(0)``,
    where E(0) is the identity element of E(A,B,FF).

    ALGORITHM: Based on the article "New algorithm for the discrete
    logarithm problem on elliptic curves" (2015) by Igor Semaev,
    available at https://eprint.iacr.org/2015/310.pdf.

    EXAMPLES::
    
    p = 19 
    
    FF = GF(p)
    
    E = EllipticCurve(FF, [0,0,0,11,1]) 
    
    print(E)
    
    SMP3 = construct3rdSumPoly(E)
    
    print(SMP3)
    
    sage: Elliptic Curve defined by y^2 = x^3 + x + 11 over Finite Field of size 19
    
    sage: x1^2*x2^2 - 2*x1^2*x2*x3 - 2*x1*x2^2*x3 + x1^2*x3^2 
    - 2*x1*x2*x3^2 + x2^2*x3^2 - 2*x1*x2 - 2*x1*x3 - 2*x2*x3
    - 6*x1 - 6*x2 - 6*x3 + 1
    """
    if E.a1() != 0 or E.a2() != 0 or E.a3() != 0:
        raise TypeError('Provided elliptic curve is not in the short Weierstrass form.') 
    
    FF = E.base_field()
    A = E.a4()
    B = E.a6()
    PR.<s1, s2, s3> = PolynomialRing(FF, 3, order='degrevlex')
    smp = (s1 - s2)**2*s3**2 - 2*((s1 + s2)*(s1*s2 + A) + 2*B)*s3 + (s1*s2 - A)**2 - 4*B*(s1 + s2)
    return smp

def calcFactorBaseSize(curveOrder, m):
    """
    returns the ideal factor base size as mentioned in https://eprint.iacr.org/2017/609.pdf
    
    INPUT:

    - ``curveOrder`` - order of the elliptic curve

    - ``m`` - decomposition constant
    
    OUTPUT: ideal factor base size (positive integer)
    
    """    
    
    return ceil(curveOrder**(1.0/m))

def solveECDLP(a, b, curveOrder):
    """
    Solves the ECDLP from the relations ``a``P + ``b``Q = curve identity.
    Find the integer ``k`` such that ``kP = Q``.
    
    INPUT:
    
    ``a`` - coefficient of the base point ``P``
    
    ``b`` - coefficient of the second point ``Q``
    
    ``curveOrder`` - order of the group generated by ``P``
    
    OUTPUT:
    
    Solution to the ECDLP, integer ``k`` such that ``kP = Q``.
    """
    a = int(a)
    b = int(b)
    res = None
    try:
        res = int(mod(-a*inverse_mod(b, curveOrder), curveOrder))
    except:
        print("Error in solveECDLP: a = {}, b = {}, curveOrd = {}.".format(a,b,curveOrder))
        pass
    return res


def buildSumPolySystem(FF, SM3, m, Rx = False):
    #number of bounding variables 'U'
    numBoundVars = m - 3
    if Rx == True: #last summation polynomial will be S_3(x_m, u_(m-2), Rx)
        numBoundVars += 1
    SMPR = PolynomialRing(FF, 'x', m + numBoundVars, order='degrevlex')
    
    #X-variables
    variablesX = SMPR.objgens()[1][0:m]
    #bounding variables
    variablesU = SMPR.objgens()[1][m:]
    
    generators = [] #add the sumPoly eqs
    for k in range(0, numBoundVars):
        if k != 0:
            generators.append(SM3(variablesU[k - 1], variablesU[k], variablesX[k + 1]))
        else:
            generators.append(SM3(variablesX[0], variablesX[1], variablesU[0]))        
    
    #we only need one summation polynomial - the last one
    if len(variablesU) == 0:
        variablesU = [variablesX[0]]
    return generators, variablesX, variablesU, SMPR


def buildFactorBaseOnlyX(Q, P, curveOrder, m):
    """

    Builds a factor base (each point has a known decomposition into ``P`` and ``Q``), 
    storing only x-coordinates.
    
    
    """
    import random
    
    factorBaseSize = calcFactorBaseSize(curveOrder, m)
    factorBase = {} #dict x-coord, index in coord
    coord = []
    
    res = None
    currentSize = 0
    ident = 0*P
    while currentSize < factorBaseSize:
        a = int(random.random() * curveOrder) #a,b in [0, orderE - 1]
        b = int(random.random() * curveOrder)
        candidate=(a*P + b*Q)
        candX = candidate[0] #store only x-coordinates
        if candidate == ident and b != 0: #we can solve the ECDLP
            res = solveECDLP(a, b, curveOrder)
            if res != None:
                break
        elif candX not in factorBase:
            factorBase[candX] = currentSize #store id to the ``coord`` list
            coord.append((a,b))
            currentSize += 1
        else: #there exists an relation aP + bQ = +-(cP + dQ)
            c,d = coord[factorBase[candX]]

            if (c*P + d*Q) != candidate:
                c = -c
                d = -d
            res = solveECDLP(a-c, b-d, curveOrder)
            if res != None:
                break
            
    return coord, factorBase, res

def generateRandomEC(bits = 0, p = 0, primeOrder=False):
    """
    Generates a random elliptic curve defined by the short
    Weierstrass equation over a prime order field.

    INPUT:
    
    - ``bits`` - number of bits of the order (characteristic) of the underlying field
    
    - ``p`` - characteristic of the prime field, if set - must be a positive prime integer

    - ``primeOrder`` - whether we care about the order of the 
                       elliptic curve to be prime
    
    Parameter ``p`` is preferred if set to ``bits``.
    
    We first construct a finite field ``FF`` of prime order with specified
    bit security. ``log[2, #FF]`` =~ ``bits``. Then generate random
    coefficients ``a_4`` and ``a_6`` of the generated elliptic curve.
    We check whether it's a non-singular curve. 
    
    In Sage this elliptic curve can be constructed this way 
    (using the long Weierstrass equation):

    E = EllipticCurve(FF, [0,0,0,a_4,a_6])
    
    OUTPUT: Returns the generated elliptic curve and the prime
    order finite field ``FF``. 
    
    EllipticCurve(FF, [0,0,0,a_4,a_6]), where ``a_4`` and ``a_6``
    are random integers. If we require the elliptic curve of
    prime order, we repeat the generation until we one is found,
    if there isn't a prime order EC found, we return an EC
    with the biggest prime factor found.

    EXAMPLES::
    
    b = 11 
    
    E, FF = generateRandomEC(bits=b, primeOrder=True)
    
    print(E) 
    
    print("Order of E is: {}.".format(E.order())) 
    
    sage: Elliptic Curve defined by y^2 = x^3 + 854*x + 303 over Finite Field of size 2053 
    
    sage: Order of E is: 2029.
    
    E, FF = generateRandomEC(p=6421, primeOrder=True)
    
    print(E)
    
    sage: Elliptic Curve defined by y^2 = x^3 + 252*x + 5611 over Finite Field of size 6421
    """
    if (p == 0):
        p = next_prime(int(2**bits));
        
    T = GF(p)
    
    coefs = [0, 0, 0, None, None]
    while True:
        #random a_4
        coefs[3] = T.random_element();
        #random a_6
        coefs[4] = T.random_element();
        try:
            E = EllipticCurve(T, coefs)
            if primeOrder == False or is_prime(E.order()):
                break
        except ArithmeticError: #if E singular we try again
            pass
    return E

def initECDLP(E):
    """
    Initializes an elliptic curve discrete logarithm
    problem on the selected elliptic curve  ``E``.

    INPUT:

    - ``E`` - ellitic curve on which the problem will be initialized
    
    ECDLP in ``E`` with a generator ``P``, and some other point ``Q``
    is to find an integer ``k`` such that ``kP`` = ``Q``.

    OUTPUT: Returns the generator ``P`` of ``E`` and a random
    point ``Q`` on ``E``. ``Q`` is guaranteed
    to be different to the elliptic curve's ``E`` identity element.

    EXAMPLES::
    
    E = EllipticCurve(GF(17), [0,0,0,1,2])
    
    P, Q = initECDLP(E)
    
    print(P,Q)
    
    sage: Order of P is 24.
    
    sage: ((13 : 11 : 1), (0 : 6 : 1))
    """
    
    import random
    
    P=E.gen(0)
    orderP=P.order()
    #k in [1, orderP-1] => Q != E(0)
    k=int(1+(orderP-1)*random.random())
    Q=k*P
    return P, Q


def boundVariablesToBase(FF, generators, variablesX, factorBase, m):
    FBR.<X> = PolynomialRing(FF, 1, order='degrevlex')
    baseBound = [FBR(1) for k in range(m)]
    
    #divide factorBase into m sets and create m-bounding polynomials
    for elem, k in factorBase.iteritems():
        baseBound[mod(k, m)] *= FBR(X - elem) #add to the m-th base  
    
    for k in range(0,m):
        generators.append(baseBound[k](variablesX[k]))

    return generators


#based on (Our Algorithm in http://www.science.unitn.it/~sala/events2016/AlessandroAmadori-BunnyTN7.pdf)
def sumPolySplitKnownDecomp(Q, P, m):
    import time
    startTime = time.time()
    
    if m == 2:
        m = 3
    if m < 3:
        raise ValueError("Parameter 'm' has to be at least 3.")
    
    #Elliptic curve
    E = P.curve()
    
    #curveOrder - assuming <P> = E
    curveOrder = P.order()
    
    #base (prime) field
    FF = E.base_field()
    
    #3rd Semaev summation polynomial for this curve
    SM3 = construct3rdSumPoly(E)
    
    #factor base ring
    ident = E(0);
    
    #build a summation polynomial system to stand-in for m-th summation polynomial
    generators, variablesX, variablesU, SMPR = buildSumPolySystem(FF, SM3, m, Rx = False)
    generators.append(SM3(variablesX[-2], variablesX[-1], variablesU[-1]))
    
    it = 0
    res = None
    while res == None: #not solved
        
        #random factorBase with known decomposition (coord)
        coord, factorBase, res = buildFactorBaseOnlyX(Q, P, curveOrder, m)
        if res != None:
            break #break if already solved
        
        #keep only the summation polynomials and renew base bounds
        generators = generators[:m-2]
        
        #restrict results to factor base only
        generators = boundVariablesToBase(FF, generators, variablesX, factorBase, m)
        
        print("Iter: {} - base of size: {} generated, m = {}. Looking for a relation now."\
              .format(it, len(factorBase), m))
        sys.stdout.flush()
        
        tmp = SMPR.ideal(generators) 

    #    gb = Ideal(tmp.groebner_basis('libsingular:slimgb')) #good
            
        gb = Ideal(tmp.groebner_basis('giac:gbasis', prot=False)) 
        
        #use FGLM to convert grevlex to lex
     #   gb = Ideal(gb.transformed_basis('fglm',LEXSMPR))
    
 #       import fgb_sage
        #Faugere implementation - for prime fields < 2^16
#        gb = fgb_sage.groebner_basis(tmp)  #68s / 107s - 37 rel: 15.9 bits of security
        
        try:
            solutions = gb.variety()
        except ValueError: #non-zero ideal
            continue
            
        for solution in solutions:
            points = [E.lift_x(solution[variablesX[k]]) for k in range(m)]
            
            for v in VectorSpace(GF(2), m):
                if sum(-points[k] if v[k] else points[k] for k in range(m)) == ident:
                    sumA = 0
                    sumB = 0
                    
                    for k in range(m):
                        baseId = factorBase[points[k][0]]
                        sgn = -1 if v[k] else 1

                        if (coord[baseId][0]*P + coord[baseId][1]*Q) == (sgn*points[k]):
                            sumA -= coord[baseId][0]
                            sumB -= coord[baseId][1]
                        else: 
                            sumA += coord[baseId][0]
                            sumB += coord[baseId][1]
                    res = solveECDLP(sumA, sumB, curveOrder)
    
        it += 1

    printResults(E, P, Q, res, curveOrder, m, len(factorBase), it, time.time() - startTime)
    return res

# def findPoints(xs, E):
#     relation = []
#     points = []
#     ident = E(0) #infinity point (identity)
#     xs = [xs["x1"], xs["x2"], xs["x3"]]
#     for val in xs:
#         if E.is_x_coord(val): #valid point
#             adept = E.lift_x(val, all=False, extend=False)
#             if adept != ident: #no new information
#                 points.append(adept)
#     #find signs
#     dim = len(points)
#     if dim > 1:
#         for v in VectorSpace(GF(2), dim - 1):
#             if (points[-1] + sum(-points[k] if v[k] else points[k] for k in range(0, dim - 1))) == ident:
#                 relation = [-points[k] if v[k] else points[k] for k in range(0, dim - 1)]
#                 relation.append(points[-1])
#                 break
#     return relation


# #based on (Out Algorithm) http://www.science.unitn.it/~sala/events2016/AlessandroAmadori-BunnyTN7.pdf
# def solveECDLP2(E, SM3, P, Q, FF, orderP, coord, factorBase):
#     PR.<x1, x2, x3> = PolynomialRing(FF,3, order='degrevlex')
#     PR1.<X> = PolynomialRing(FF,1, order='degrevlex')
#     res = -1
#     factorBaseSize = len(factorBase)
    
#     ident = E(0) #EC identity element
    
#     #generators should be: SM3(x1, x2, x3)
#     #we can add additional contrains (instead of 3 cycles):
#     #product(X - X1) = 0 => X1 is in the factorBase
#     #product(X - X2) = 0 => X2 is in the factorBase
#     #product(X - X3) = 0 => X3 is in the factorBase
#     generators = [SM3(x1,x2,x3)]
    
#     genPoly = product((X - factorBase[k]) for k in srange(0, factorBaseSize))
#     generators.append(genPoly(x1))
#     generators.append(genPoly(x2))
#     generators.append(genPoly(x3))
#     #solve
#     tmp = PR.ideal(generators);
#     gb = tmp.groebner_basis('libsingular:groebner')
#     variety = (PR.ideal(gb)).variety()
#     for solution in variety:
#         relation = findPoints(solution, E)
#         sumA = 0
#         sumB = 0
#         for rel in relation:
#             if rel[0] in factorBase:
#                 baseId = factorBase.index(rel[0])
#                 if ( (coord[baseId][0]*P + coord[baseId][1]*Q) == rel ):
#                     sumA -= coord[baseId][0]
#                     sumB += coord[baseId][1]
#                 else: # (-rel) is in factorBase
#                     sumA += coord[baseId][0]
#                     sumB -= coord[baseId][1]
#             else:
#                 print("something fishy...")
#         try:
#             res = Integer(mod(sumA*inverse_mod(sumB, orderP), orderP))
#             if res*P == Q: #is it really the solution?
#                 break
#             else:
#                 print("Found {} is not really a solution, something went wrong.".format(res))
#         except (ZeroDivisionError ,NotImplementedError): #inverse of 'sumB' mod 'orderP' does not exist
#             print("SumB = {}, SumA = {}, not invertible mod orderP ({}).".format(sumB, sumA, orderP))
#             pass
                        
#     return res

def sumPolyOrigApproachSplitGeneral(Q, P, m):
    import random
    import time
    
    startTime = time.time()
    if m < 2:
        raise ValueError("Parameter 'm' has to be at least 2.")
    
    #Elliptic curve
    E = P.curve()
    
    #3rd Semaev summation polynomial for 'E'
    SM3 = construct3rdSumPoly(E) 
    
    #finite (prime) field
    FF = E.base_field()
    
    #order of <P>, assuming E = <P>
    orderP = P.order() 
    
    #relation matrix field - orderP is prime
    CF = GF(orderP); 
    
    #factorBase size
    fbs = calcFactorBaseSize(orderP, m)
    
    FFmid = float(FF.order())/2.0;
    
    #max rank is fbs + 1, use Sparse matrix for effective computation
    relationMatrix = matrix(CF, nrows=fbs+1, ncols=fbs+2, sparse=True); 
    
    #build a summation polynomial system to stand-in for m-th summation polynomial
    generators, variablesX, variablesU, SMPR = buildSumPolySystem(FF, SM3, m, Rx = True)

    columnID = 0 
    factorBase = {}
    while columnID < fbs:
        # generate a random x-coordinate
        candX = int(1 + random.random()*(FF.order() - 1)) 
        if candX not in factorBase.keys() and E.is_x_coord(candX):
            factorBase[candX] = columnID
            columnID += 1

    #restrict results to the factor base
    generators = boundVariablesToBase(FF, generators, variablesX, factorBase, m)

    #we will replace this with SM3(u_{m-2}, x_m, R_x)
    generators.append(None) 
    
    ident = E(0);
    numRel = 0; #number of relations
    print("Base of size: {} generated. Entering the main cycle.".format(fbs))
    sys.stdout.flush()

    isItEnough = False;
    res = None
    totalRel = 0
    
    while isItEnough == False:
        a=int(orderP * random.random())
        b=int(orderP * random.random())
        R = a*P + b*Q
        if R == ident:
            res = solveECDLP(a, b, orderP)
            if res != None:
                break
            else:
                continue

        Rx = R[0];
        generators[-1] = SM3(variablesU[-1], variablesX[-1], Rx); 
        tmp = SMPR.ideal(generators) 

       # gb = tmp.groebner_basis('libsingular:slimgb') #good
        gb = Ideal(tmp.groebner_basis('giac:gbasis', prot=False)) 

        #use FGLM to convert grevlex to lex
#       gb = Ideal(gb.transformed_basis('fglm',LEXSMPR))
    
 #       import fgb_sage
        #Faugere implementation - for prime fields < 2^16
#        gb = Ideal(fgb_sage.groebner_basis(tmp))  #68s / 107s - 37 rel: 15.9 bits of security
        try:
            solutions = gb.variety()
        except ValueError: #non-zero ideal
            continue
            
        addedRel = 0
        
        for solution in solutions:
            candidates = [E.lift_x(solution[variablesX[k]]) for k in range(m)]
            points = [candidates[k] if float(candidates[k][1]) < FFmid else -candidates[k] for k in range(m)]
            
            for v in VectorSpace(GF(2), m):
                if (R + sum(-points[k] if v[k] else points[k] for k in range(m))) == ident:
                    for k in range(m): 
                        relationMatrix[numRel, factorBase[points[k][0]]] += (-1)**(v[k])
                        
                    relationMatrix[numRel, fbs] = a
                    relationMatrix[numRel, fbs + 1] = b
                    numRel += 1
                    addedRel += 1
                    break
            if numRel >= fbs:
                if fbs not in relationMatrix.pivots(): #cant solve with those relations
                    relationMatrix = copy(relationMatrix.echelon_form()); #make mutable again
                    numRel = relationMatrix.rank()
                    print("Acquired relations aren't sufficient to solve the ECDLP. Relation matrix rank: {}"\
                          .format(numRel))
                    sys.stdout.flush()
                else:
                    break
        if addedRel > 0:
            if fbs in relationMatrix.pivots(): #we can solve the ECDLP now
                isItEnough = True
            else:
              #  print("Added {} relations to the relation matrix.".format(addedRel))
                totalRel += addedRel
                      
    if res == None: 
        relationMatrix = relationMatrix.echelon_form();
        ##in the echelon form, so the last nonzero row has to be (0 ... 0 1 c) = > P + cQ = curve identity
        res = solveECDLP(1, relationMatrix[relationMatrix.rank() - 1, fbs + 1], orderP) #0-indexed
    
    printResults(E, P, Q, res, orderP, m, fbs, totalRel, time.time() - startTime)
        
    return res
    



In [85]:
#### Tests:
bits = 13
E = generateRandomEC(bits=bits, primeOrder=True)
print(E)
sys.stdout.flush()
P, Q = initECDLP(E)

orderP = P.order()
deg = 7
for k in range(2,20):
    if calcFactorBaseSize(orderP, k) <= k*deg:
        m = k
        break

print(E)
print("orderP: {}, m = {}.".format(orderP, m))
sys.stdout.flush()
sumPolySplitKnownDecomp(Q, P, m)
#printMaxMemUsage()

sumPolyOrigApproachSplitGeneral(Q, P, m);




Elliptic Curve defined by y^2 = x^3 + 5914*x + 2608 over Finite Field of size 8209
Elliptic Curve defined by y^2 = x^3 + 5914*x + 2608 over Finite Field of size 8209
orderP: 8221, m = 3.
Iter: 0 - base of size: 21 generated, m = 3. Looking for a relation now.

Results: 
Group: Elliptic Curve defined by y^2 = x^3 + 5914*x + 2608 over Finite Field of size 8209,
#P = 8221,
P = (3876 : 5246 : 1),
Q = (3882 : 1491 : 1),
res = 8043,
m = 3,
Factor base size = 21,
Number of attempts = 1,
Total time = 0.104120969772 seconds,
Result is valid = True.
Base of size: 21 generated. Entering the main cycle.

Results: 
Group: Elliptic Curve defined by y^2 = x^3 + 5914*x + 2608 over Finite Field of size 8209,
#P = 8221,
P = (3876 : 5246 : 1),
Q = (3882 : 1491 : 1),
res = 8043,
m = 3,
Factor base size = 21,
Number of attempts = 8,
Total time = 1.80058002472 seconds,
Result is valid = True.


In [61]:
#Individual step of Pohlig-Hellman algorithm
#Solves DLP in a subgroup of order 'orderP'
#https://courses.fit.cvut.cz/MI-MKY/media/lectures/mi-mky-poznamky-v17.pdf (page 81 PDF)
#Pi - generator of the subgroup, Qi = ki*Pi, order of Pi is orderP, returns ki
#Splits ki = x0 + p*x1 + p^2*x2 + p^(e-1)*x_(e-1) and solves 'e'-times DLP in a subgroup of order 'orderP'
def PH_reduction(Qi, Pi, orderP, **kwargs): 
    invPi = -Pi #additive inverse
    p = prime_factors(orderP)[0]
    e = orderP.valuation(p)
    
    Pi = (p**(e-1))*Pi
    ki = 0 #ki = x0 + p*x1 + p^2*x2 + ...
    for i in srange(0, e):
        tempQ = (p**(e-1-i))*(Qi + ki*invPi)
        ki += (p**i)*discrete_log_rho(tempQ, Pi, operation="+")
    return ki


#Implements Pohlig-Hellman algorithm
#https://courses.fit.cvut.cz/MI-MKY/media/lectures/mi-mky-poznamky-v17.pdf (page 81 PDF)
#Solves Q = k*P, returns k
#P generates a subgroup of order orderP
#DLP_Solve is a function solving ECDLP in a subgroup of order p^e, where 'p' is a prime number
def Pohlig_Hellman_additive(Q, P, orderP, DLP_Solve = PH_reduction, **kwargs):
    print("Factorization of orderP: {} = {}.".format(orderP, orderP.factor()))
    print("Order of E = <P> is {}.\nBit security (log2 of the biggest prime_factor of orderP): {:4.1f}"\
      .format(orderP, float(log(max(prime_factors(orderP)))/log2)))
    
    ki = [] #list of individual DLP results
    mi = [] #list of moduli
    for p in prime_factors(orderP): #prime factor p
        e = orderP.valuation(p) #multiplicity of p in orderP
        mi.append(p**e) 
        pei = Integer(orderP/mi[-1])
        ki.append(DLP_Solve(pei*Q, pei*P, mi[-1], **kwargs)) #solves ECDLP in a subgroup <pei*P>
    return Integer(CRT(ki, mi))

In [ ]:
#original algorithm based on summation polynomials
#choose parameter "m"
#choose factor base as points with x-coordinate <= b = ceil(p/m), store only x-coord
#randomly generate point R = aP + bQ
#try to express it as a combination of factor base points => S_{m+1}(x_1, \ldots, x_m, R_x), x_i \in factorbase
#if successfull store it as a relation in a matrix with b + 2 columns, +1 (point with lower y-coord), -1 (point with higher y-cord) 
#(first b are for the base points and last two are a,b coefficinets respectively)
#collect as many relationship to be able to create a row (0 ... 0 a b) and solve ECDLP
from sage.misc.superseded import experimental
@experimental(66666)
def sumPolyOrigApproach(P, Q):
    from random import random
    E = P.curve() #Elliptic curve
    SM3 = construct3rdSumPoly(E) #3rd Semaev summation polynomial
    FF = E.base_field() #base field
    
    orderP = P.order() #order of <P>
    CF = GF(orderP); #relation matrix field
    fbs = ceil(orderP**(1/3)); #factor base size
    
    FFmid = float(FF.order())/2.0;
    #max rank is fbs + 1, use Sparse for effective computation
    relationMatrix = matrix(CF, nrows=fbs+1, ncols=fbs+2, sparse=True); 
    
    #factor base ring
    FBR.<X> = PolynomialRing(FF,1, order='degrevlex') 
    #summation poly ring
    SMPR.<x1,x2,x3,u1> = PolynomialRing(FF,4, order='degrevlex')
    baseBound = FF(1) #baseBound is a product(X - x_i), x_i in factorBase
    
    colId = 0
    #map numbers in FF to columns in relationMatrix
    colIdDict = {}
    for i in range(0, fbs + 1):
         #generate random basis
        candidate = int(1 + random()*(orderP - 1))
        if candidate not in colIdDict.keys() and E.is_x_coord(candidate):
            baseBound *= (X - candidate) #add to the factor base
            colIdDict[candidate] = colId;
            colId += 1
    
    #bound all x_1,x_2,x_3 to be in factorBase and prepare a binding variable u_1
    generators = [baseBound(x1), baseBound(x2), baseBound(x3), SM3(x1,x2,u1), None]
    ident = E(0);
    relNum = 0; #number of relations
    print("Base generated: entering the main cycle.")
    sys.stdout.flush()
    isItEnough = False;
    res = None
    
    while isItEnough == False:
        a=int(orderP * random())
        b=int(orderP * random())
        R = a*P + b*Q
        if R == ident:
            if b != 0:
                res = CF(-a)*CF(b)**(-1)
                break
            else:
                continue

        Rx = R[0];
        generators[4] = SM3(u1,x3,Rx);
        tmp = SMPR.ideal(generators);
        gb = tmp.groebner_basis('libsingular:groebner')
        solutions = (SMPR.ideal(gb)).variety()
        for solution in solutions:
            candidates = [E.lift_x(solution["x1"]), E.lift_x(solution["x2"]), E.lift_x(solution["x3"])]
            points = [candidates[k] if float(candidates[k][1]) < FFmid else -candidates[k] for k in [0,1,2]]
            
            for v in VectorSpace(GF(2), 3):
                if (R + sum(-points[k] if v[k] else points[k] for k in range(0, 3))) == ident:
                      
                    for k in [0,1,2]:
                        relationMatrix[relNum, colIdDict[points[k][0]]] += (-1)**(v[k]);
                    relationMatrix[relNum, fbs] = a;
                    relationMatrix[relNum, fbs + 1] = b;
                    relNum += 1;
                    break
            if relNum >= fbs:
                if fbs not in relationMatrix.pivots(): #cant solve with those relations
                    relationMatrix = copy(relationMatrix.echelon_form()); #make mutable again
                    relNum = relationMatrix.rank()
                    print("Acquired relations aren't sufficient to solve the ECDLP. \\
                          Relation matrix rank: {}".format(relNum))
                    sys.stdout.flush()
                break
        if len(solutions) > 0:
            if fbs in relationMatrix.pivots(): #we can solve the ECDLP now
                isItEnough = True
            else:
                print("Added {} relations to the relation matrix.".format(len(solutions)))
                      
    print(relationMatrix)
    print("Echelon form:\n")
    relationMatrix = relationMatrix.echelon_form();
    print(relationMatrix)
    res = (-relationMatrix[relationMatrix.rank() - 1, fbs + 1])**(-1)
    print("Found solution: P = {}, Q = {}, k = {}, isItCorrect: {}"
          .format(P, Q, res, int(res)*P == Q))
    
#### Tests:
import time
bits = 15
E = generateRandomEC(bits, True)
print(E)
sys.stdout.flush()
P, Q = initECDLP(E)
tm = time.time()
sumPolyOrigApproach(P, Q)
print("It took {} seconds.".format(time.time() - tm))

In [ ]:
#original algorithm based on summation polynomials
#choose parameter "m"
#choose factor base as points with x-coordinate <= b = ceil(p/m), store only x-coord
#randomly generate point R = aP + bQ
#try to express it as a combination of factor base points => S_{m+1}(x_1, \ldots, x_m, R_x), x_i \in factorbase
#if successfull store it as a relation in a matrix with b + 2 columns, +1 (point with lower y-coord), -1 (point with higher y-cord) 
#(first b are for the base points and last two are a,b coefficinets respectively)
#collect as many relationship to be able to create a row (0 ... 0 a b) and solve ECDLP
from sage.misc.superseded import experimental
@experimental(66666)
def sumPolyOrigApproachSplit(P, Q):
    from random import random
    E = P.curve() #Elliptic curve
    SM3 = construct3rdSumPoly(E) #3rd Semaev summation polynomial
    FF = E.base_field() #base field
    
    orderP = P.order() #order of <P>
    CF = GF(orderP); #relation matrix field
    fbs = ceil(orderP**(1/3)); #factor base size
    
    FFmid = float(FF.order())/2.0;
    #max rank is fbs + 1, use Sparse for effective computation
    relationMatrix = matrix(CF, nrows=fbs+1, ncols=fbs+2, sparse=True); 
    
    #factor base ring
    FBR.<X> = PolynomialRing(FF,1, order='degrevlex') 
    #summation poly ring
    SMPR.<x1,x2,x3,u1> = PolynomialRing(FF,4, order='degrevlex')
    baseBound1 = FF(1) #baseBound is a product(X - x_i), x_i in F1
    baseBound2 = FF(1) #baseBound is a product(X - x_i), x_i in F2
    baseBound3 = FF(1) #baseBound is a product(X - x_i), x_i in F3
    
    colId = 0
    #map numbers in FF to columns in relationMatrix
    colIdDict = {}
    for i in range(0, fbs + 1):
         #generate random basis
        candidate = int(1 + random()*(orderP - 1))
        if candidate not in colIdDict.keys() and E.is_x_coord(candidate):
            if mod(colId, 3) == 0:
                baseBound1 *= (X - candidate) #add to the factor base F1
            elif mod(colId,3) == 1:
                baseBound2 *= (X - candidate) #add to the factor base F2
            else: 
                baseBound3 *= (X - candidate) #add to the factor base F3
          
            colIdDict[candidate] = colId;
            colId += 1
    
    #bound all x_1,x_2,x_3 to be in factorBase and prepare a binding variable u_1
    generators = [baseBound1(x1), baseBound2(x2), baseBound3(x3), SM3(x1,x2,u1), None]
    ident = E(0);
    relNum = 0; #number of relations
    print("Base generated: entering the main cycle.")
    sys.stdout.flush()
    isItEnough = False;
    res = None
    
    while isItEnough == False:
        a=int(orderP * random())
        b=int(orderP * random())
        R = a*P + b*Q
        if R == ident:
            if b != 0:
                res = CF(-a)*CF(b)**(-1)
                break
            else:
                continue

        Rx = R[0];
        generators[4] = SM3(u1,x3,Rx);
        tmp = SMPR.ideal(generators);
        gb = tmp.groebner_basis('libsingular:groebner')
        solutions = (SMPR.ideal(gb)).variety()
        for solution in solutions:
            candidates = [E.lift_x(solution["x1"]), E.lift_x(solution["x2"]), E.lift_x(solution["x3"])]
            points = [candidates[k] if float(candidates[k][1]) < FFmid else -candidates[k] for k in [0,1,2]]
            
            for v in VectorSpace(GF(2), 3):
                if (R + sum(-points[k] if v[k] else points[k] for k in range(0, 3))) == ident:
                      
                    for k in [0,1,2]:
                        relationMatrix[relNum, colIdDict[points[k][0]]] += (-1)**(v[k]);
                    relationMatrix[relNum, fbs] = a;
                    relationMatrix[relNum, fbs + 1] = b;
                    relNum += 1;
                    break
            if relNum >= fbs:
                if fbs not in relationMatrix.pivots(): #cant solve with those relations
                    relationMatrix = copy(relationMatrix.echelon_form()); #make mutable again
                    relNum = relationMatrix.rank()
                    print("Acquired relations aren't sufficient to solve the ECDLP. \\
                          Relation matrix rank: {}".format(relNum))
                    sys.stdout.flush()
                break
        if len(solutions) > 0:
            if fbs in relationMatrix.pivots(): #we can solve the ECDLP now
                isItEnough = True
            else:
                print("Added {} relations to the relation matrix.".format(len(solutions)))
                      
#    print(relationMatrix)
#    print("Echelon form:\n")
    relationMatrix = relationMatrix.echelon_form();
#    print(relationMatrix)
    res = (-relationMatrix[relationMatrix.rank() - 1, fbs + 1])**(-1)
    print("Found solution: P = {}, Q = {}, k = {}, isItCorrect: {}"
          .format(P, Q, res, int(res)*P == Q))
    
#### Tests:
import time
bits = 21
E = generateRandomEC(bits, True)
print(E)
sys.stdout.flush()
P, Q = initECDLP(E)
tm = time.time()
sumPolyOrigApproachSplit(P, Q)
print("It took {} seconds.".format(time.time() - tm))

In [67]:
#original algorithm based on summation polynomials
#choose parameter "m"
#choose factor base as points with x-coordinate <= b = ceil(p/m), store only x-coord
#randomly generate point R = aP + bQ
#try to express it as a combination of factor base points => S_{m+1}(x_1, \ldots, x_m, R_x), x_i \in factorbase
#if successfull store it as a relation in a matrix with b + 2 columns, +1 (point with lower y-coord), -1 (point with higher y-cord) 
#(first b are for the base points and last two are a,b coefficinets respectively)
#collect as many relationship to be able to create a row (0 ... 0 a b) and solve ECDLP
#from sage.misc.superseded import experimental
#@experimental(66666)
import giacpy_sage
giacpy_sage.giacsettings.threads=4
def sumPolyOrigApproachSplitGeneral(P, Q, m = 3):
    from random import random
    if m < 2:
        raise ValueError("Parameter 'm' has to be at least 2.")
    E = P.curve() #Elliptic curve
    SM3 = construct3rdSumPoly(E) #3rd Semaev summation polynomial
    FF = E.base_field() #base field
    
    orderP = P.order() #order of <P>
    CF = GF(orderP); #relation matrix field
    fbs = ceil(orderP**(1/m)); #factor base size
    
    FFmid = float(FF.order())/2.0;
    #max rank is fbs + 1, use Sparse for effective computation
    relationMatrix = matrix(CF, nrows=fbs+1, ncols=fbs+2, sparse=True); 
    
    #factor base ring
    FBR.<X> = PolynomialRing(FF, 1, order='degrevlex') 
    #summation poly ring
    #last m - 2 are in fact u_i
    SMPR = PolynomialRing(FF, 'x', m + (m - 2), order='degrevlex')
#    LEXSMPR = PolynomialRing(FF, 'x', m + (m - 2), order='lex')
    variablesX = SMPR.objgens()[1][0:m]
    variablesU = SMPR.objgens()[1][m:]
    baseBound = [FBR(1) for k in range(m)]
        
    colId = 0
    #dict to map numbers in FF to columns in relationMatrix
    colIdDict = {}
    while colId < fbs:
         #generate random basis
        candidate = int(1 + random()*(FF.order() - 1)) ## generating an x-coordinate
        if candidate not in colIdDict.keys() and E.is_x_coord(candidate):
            baseBound[mod(colId, m)] *= FBR(X - candidate) #add to the m-th base          
            colIdDict[candidate] = colId
            colId += 1
    
    #bound all x_1,x_2,x_3 to be in factorBase and prepare a binding variable u_1
    generators = []
    for k in range(0,m):
        generators.append(baseBound[k](variablesX[k]))
    
    for k in range(0, m - 2):
        if k != 0:
            generators.append(SM3(variablesU[k - 1], variablesU[k], variablesX[k + 1]))
        else:
            generators.append(SM3(variablesX[0], variablesX[1], variablesU[0]))
    generators.append(None)
    ident = E(0);
    relNum = 0; #number of relations
    print("Base of size: {} generated. Entering the main cycle.".format(fbs))
    sys.stdout.flush()
    isItEnough = False;
    res = None
    
    if m == 2: #hotFix, we need just 1 summation polynomial
        variablesU = [variablesX[0]];
        
    libsg = 0
    fgm = 0
    guac = 0
    lstd = 0
    lslimgb = 0
    lstdhilb = 0
    lstdfglm = 0

    while isItEnough == False:
        a=int(orderP * random())
        b=int(orderP * random())
        R = a*P + b*Q
        if R == ident:
            if b != 0:
                res = CF(-a)*CF(b)**(-1)
                break
            else:
                continue

        Rx = R[0];
        generators[-1] = SM3(variablesU[-1], variablesX[-1], Rx); 
        tmp = SMPR.ideal(generators) #+ sage.rings.ideal.FieldIdeal(SMPR);

#         tm = time.time()
       # gb = tmp.groebner_basis('libsingular:slimgb') #good
#         lslimgb += time.time() - tm;
                
  #      tm = time.time()
        gb = Ideal(tmp.groebner_basis('giac:gbasis',prot=False,threads=4)) #95s 15.9 bits of security, 88s - 39
        sys.stdout.flush()
        #use FGLM to convert grevlex to lex
#       gb = Ideal(gb.transformed_basis('fglm',LEXSMPR))
    
    #       guac += time.time() - tm
 #       import fgb_sage
        #Faugere implementation - for prime fields < 2^16
#        tm = time.time()
#        gb = fgb_sage.groebner_basis(tmp)  #68s / 107s - 37 rel: 15.9 bits of security
 #       fgm += time.time() - tm
#        solutions = (SMPR.ideal(gb)).variety()
        solutions = gb.variety()
        for solution in solutions:
            candidates = [E.lift_x(solution[variablesX[k]]) for k in range(m)]
            points = [candidates[k] if float(candidates[k][1]) < FFmid else -candidates[k] for k in range(m)]
            
            for v in VectorSpace(GF(2), m):
                if (R + sum(-points[k] if v[k] else points[k] for k in range(m))) == ident:
                    
                    for k in range(m): 
                        relationMatrix[relNum, colIdDict[points[k][0]]] += (-1)**(v[k]);
                        
                    relationMatrix[relNum, fbs] = a;
                    relationMatrix[relNum, fbs + 1] = b;
                    relNum += 1;
                    break
            if relNum >= fbs:
                if fbs not in relationMatrix.pivots(): #cant solve with those relations
                    relationMatrix = copy(relationMatrix.echelon_form()); #make mutable again
                    relNum = relationMatrix.rank()
                    print("Acquired relations aren't sufficient to solve the ECDLP. Relation matrix rank: {}".format(relNum))
                    sys.stdout.flush()
                else:
                    break
        if len(solutions) > 0:
            if fbs in relationMatrix.pivots(): #we can solve the ECDLP now
                isItEnough = True
            else:
                print("Added {} relations to the relation matrix.".format(len(solutions)))
#                totalRel += len(solutions)
                pass
                      
    if res == None: 
#        print(relationMatrix)
#        print("Echelon form:\n")
        relationMatrix = relationMatrix.echelon_form();
#        print(relationMatrix)
        res = (-relationMatrix[relationMatrix.rank() - 1, fbs + 1])**(-1)
    print("times: std: {}, grobner: {}, slimgb: {}, stdhilb: {}, stdfglm: {}, giac: {}, fgb: {}.".format(lstd, libsg, lslimgb, lstdhilb, lstdfglm, guac, fgm))
    print("Found solution: P = {}, Q = {}, k = {}, isItCorrect: {}.".format(P, Q, res, int(res)*P == Q))
    return res
    
    
#### Tests:
import time
bits = 13
E = generateRandomEC(bits=bits, primeOrder=True)
print(E)
sys.stdout.flush()
P, Q = initECDLP(E)
tm = time.time()
sumPolyOrigApproachSplitGeneral(P, Q, 5)
print("It took {} seconds.".format(time.time() - tm))
printMaxMemUsage()

Elliptic Curve defined by y^2 = x^3 + 1898*x + 7044 over Finite Field of size 8209
Base of size: 7 generated. Entering the main cycle.
Added 1 relations to the relation matrix.
Added 1 relations to the relation matrix.
Added 1 relations to the relation matrix.
Added 1 relations to the relation matrix.
Added 1 relations to the relation matrix.
Added 1 relations to the relation matrix.
Acquired relations aren't sufficient to solve the ECDLP. Relation matrix rank: 7
Added 1 relations to the relation matrix.
times: std: 0, grobner: 0, slimgb: 0, stdhilb: 0, stdfglm: 0, giac: 0, fgb: 0.
Found solution: P = (213 : 7727 : 1), Q = (7879 : 7234 : 1), k = 5084, isItCorrect: True.
It took 5.39503383636 seconds.
Maximum memory usage during this session was: 203.921875 MegaBytes.


In [ ]:
import argparse
import logging

parser = argparse.ArgumentParser()
parser.add_argument(
    '-d', '--debug',
    help="Print lots of debugging statements",
    action="store_const", dest="loglevel", const=logging.DEBUG,
    default=logging.WARNING,
)
parser.add_argument(
    '-v', '--verbose',
    help="Be verbose",
    action="store_const", dest="loglevel", const=logging.INFO,
)
args = parser.parse_args()    
logging.basicConfig(level=args.loglevel)


In [ ]:
import time
import random

def tt(a):
    print("called tt", a)
    return True;
cf = matrix(ZZ, ncols=11, nrows=10, sparse=True)
cf[2,5] = 17
cf[4,3] = 7
cf[4,4] = 217
cf[4,5] = 227
cf = copy(cf.echelon_form());

cf[4,5] = 227

if 10 > 20 and tt(10):
    print("inside")



In [ ]:
E = EllipticCurve(GF(7),[0,0,0,1,3])
FF = E.base_field()
P = E.gen(0)
print(E.base_field())
print(E.a6())

In [ ]:
SMPR = PolynomialRing(GF(19), "x", 4, order='degrevlex')
variables = SMPR.objgens()[1]
print(variables[0:-1])

In [ ]:
SMPR = PolynomialRing(GF(19), "x", 2, order='degrevlex')
variables = SMPR.objgens()[1]
generators = [(17 - variables[0]), (12- variables[1])]
tmp = SMPR.ideal(generators);
gb = tmp.groebner_basis('libsingular:groebner')
solutions = (SMPR.ideal(gb)).variety()
for sol in solutions:
    print(sol[variables[0]])

In [ ]:
{769: 14, 238: 21, 407: 9, 282: 12, 929: 20, 548: 18, 679: 10, 944: 7, 561: 23, 825: 22, 442: 11, 190: 0, 320: 5, 193: 1, 195: 3, 456: 24, 843: 13, 718: 30, 720: 6, 724: 19, 598: 2, 57: 25, 994: 32, 611: 29, 613: 16, 876: 28, 749: 31, 1006: 4, 245: 26, 511: 8, 127: 15, 636: 27, 1045: 17}
[(282 : 370 : 1), (14 : 210 : 1)]
[T^17 + 382*T^16 + 792*T^15 + 1009*T^14 + 811*T^13 + 184*T^12 + 414*T^11 + 359*T^10 + 1005*T^9 + 8*T^8 + 539*T^7 + 844*T^6 + 643*T^5 + 384*T^4 + 108*T^3 + 472*T^2 + 981*T + 512, T^16 + 156*T^15 + 170*T^14 + 943*T^13 + 525*T^12 + 897*T^11 + 207*T^10 + 383*T^9 + 63*T^8 + 223*T^7 + 483*T^6 + 258*T^5 + 1005*T^4 + 798*T^3 + 947*T^2 + 641*T + 528]

In [ ]:
PR.<T>=PolynomialRing(GF(1031))
pl=T^16 + 156*T^15 + 170*T^14 + 943*T^13 + 525*T^12 + 897*T^11 + 207*T^10 + 383*T^9 + 63*T^8 + 223*T^7 + 483*T^6 + 258*T^5 + 1005*T^4 + 798*T^3 + 947*T^2 + 641*T + 528
print(pl.roots())

In [ ]:
print(construct3rdSumPoly(EllipticCurve(GF(71),[0,0,0,5,5])))

In [ ]:
PR.<x1, x2, x3, A, B, y1,y2> = PolynomialRing(ZZ, 7, order='degrevlex')
tmp = (x2-x1)^2*x3^2-2*((x1+x2)*(x1*x2 + A) + 2*B)*x3 + (x1*x2-A)^2 - 4*B*(x1+x2)

#rt = ((3*x1^2+A)^2-8*x1*(x1^3+A*x1+B))
#4*(x1^3+A*x1+B)
rt = ((-2*x1)*4*(x1^3+A*x1+B))
print(expand(rt))
print(latex(expand(rt)))
# tmp = (x1^2*x2 + x1*x2^2 + A*x1+ A*x2 + 2*y1*y2 + 2*B)*(x1^2*x2 + x1*x2^2 + A*x1 + A*x2 - 2*y1*y2 + 2*B)
# print(expand(tmp))
# print()
# print(latex(expand(tmp)))
# tmp2 = x1^4*x2^2 + 2*x1^3*x2^3 + x1^2*x2^4 + 2*x1^3*x2*A + 4*x1^2*x2^2*A + 2*x1*x2^3*A + x1^2*A^2 + 2*x1*x2*A^2 + x2^2*A^2 + 4*x1^2*x2*B + 4*x1*x2^2*B - 4*(x1^3+A*x1+B)*(x2^3+A*x2+B) + 4*x1*A*B + 4*x2*A*B + 4*B^2
# print()
# print(expand(tmp2))
# print()
# print(latex(expand(tmp2)))

# # #print()
# # #print(tmp2.simplify_full())


# # otherWay = ((x1*x2-A)^2-4*B*(x1+x2))*(x1-x2)^2
# # print()
# # print(expand(otherWay))
# # print()
# # print(latex(expand(otherWay)))

# print()
# print(latex(expand((x1*x2-A)^2-4*B*(x1+x2))))


In [7]:
import fgb_sage

PR.<X>=PolynomialRing(GF(5),1)
tmp = PR.ideal([X**2 + X, 2*X])
#tmp.groebner_basis('giac:gbasis') #95s 15.9 bits of security, 88s - 39
gb = fgb_sage.groebner_basis(tmp)
print(gb)

[X]


In [11]:
from sage.libs.giac import groebner_basis as gb_giac

P.<x,y,z>=PolynomialRing(FiniteField(29), order ='lex')
I = P.ideal(x + z*y*x + 4*z ,z + x* z + y*z , x*y*z - 1)
J = sage.rings.ideal.FieldIdeal(P)
K = I + J
print(K.
#G=gb_giac(K, prot=True, threads=3, with_f5=true)
#print(G) 

#sage : J = I + sage . rings . ideal . FieldIdeal (P)
#sage : g0 ,g1 , g2 = J. groebner_basis (); g0 ,g1 , g2
#(x + y + z , y ^2 + y*z + z ^2 , z ^3 - 1)

NotImplementedError: 

In [10]:
gb_giac?

In [5]:
1048576 - 1024**2

0

In [7]:
sage.libs.giac.groebner_basis?

Object `sage.libs.giac.groebner_basis` not found.


In [2]:
import giacpy_sage
giacpy_sage.giacsettings.threads=4
print(giacpy_sage.giacsettings.threads)

4


In [31]:
a = 19
b = 0
curveOrder = next_prime(192)
res = mod(-a*(b)**(-1), curveOrder)
print(res)
K = GF(curveOrder)
oo = -K(a)*(K(b))**(-1)
print(oo)

ZeroDivisionError: rational division by zero

In [22]:
#Error in solveECDLP: a = 842, b = -280, curveOrd = 1019.
# Error in solveECDLP: a = 357, b = 65, curveOrd = 1061.
# Iter: 2 - base of size: 11 generated, m = 3. Looking for a relation now.
# Error in solveECDLP: a = -254, b = -709, curveOrd = 1061.

F=GF(1061)
print(F(-65)**(-1)*F(357))
print(F(709)**(-1)*F(-254))

272
272


237


In [106]:
def sumPolyOrigApproachSplitGeneral2(Q, P, m = 3):
    import random
    import time
    
    startTime = time.time()
    if m < 3:
        raise ValueError("Parameter 'm' has to be at least 2.")
    
    #Elliptic curve
    E = P.curve()
    
    #3rd Semaev summation polynomial for 'E'
    SM3 = construct3rdSumPoly(E) 
    
    #finite (prime) field
    FF = E.base_field()
    
    #order of <P>, assuming E = <P>
    orderP = P.order() 
    
    #relation matrix field - orderP is prime
    CF = GF(orderP); 
    
    #factorBase size
    fbs = calcFactorBaseSize(orderP, m)
    
    FFmid = float(FF.order())/2.0;
    
    #max rank is fbs + 1, use Sparse for effective computation
    relationMatrix = matrix(CF, nrows=fbs+1, ncols=fbs+2, sparse=True); 
    
    #factor base ring
    FBR.<X> = PolynomialRing(FF, 1, order='lex') 

    #summation poly ring
    #last m - 2 are in fact u_i
    SMPR = PolynomialRing(FF, 'x', m, order='lex')
#    LEXSMPR = PolynomialRing(FF, 'x', m + (m - 2), order='lex')
    variablesX = SMPR.objgens()[1][0:m]
    variablesU = SMPR.objgens()[1][m:]
    baseBound = [FBR(1) for k in range(m)]
        
    coord = []
    #dict to map numbers in FF to columns in relationMatrix
    factorBase = {}
    colId = 0
    while colId < fbs:
        #generate random basis
        ## generating an x-coordinate
        a = int(1 + (random.random() * (orderP - 1)))
        b = int(1 + (random.random() * (orderP - 1)))
        candX = (a*P + b*Q)[0]
        if candX not in factorBase.keys() and E.is_x_coord(candX):
            baseBound[mod(colId, m)] *= FBR(X - candX) #add to the m-th base          
            factorBase[candX] = colId
            colId += 1
    
    #bound all x_1,x_2,...,x_m to be in factorBase and prepare a binding variable u_1,...,u_m-2
    generators = []
    for k in range(0,m):
        generators.append(baseBound[k](variablesX[k]))
    
    for k in range(0, m - 3):
        if k != 0:
            generators.append(SM3(variablesU[k - 1], variablesU[k], variablesX[k + 1]))
        else:
            generators.append(SM3(variablesX[0], variablesX[1], variablesU[0]))
    generators.append(SM3(variablesX[2], variablesX[1], variablesX[0])) #we will replace this with SM3(u_{m-2}, x_m, R_x)
    ident = E(0);
    numRel = 0; #number of relations
    print("Base of size: {} generated. Entering the main cycle.".format(fbs))
    sys.stdout.flush()
    
    #hotFix, for m = 2 we dont need any binding variables
    #can use just one summation polynomial
    if m == 2: 
        variablesU = [variablesX[0]];

    isItEnough = False;
    res = None
    it = 0
    while isItEnough == False:
        tmp = SMPR.ideal(generators) #+ sage.rings.ideal.FieldIdeal(SMPR);

        gb = Ideal(tmp.groebner_basis('giac:gbasis',prot=False)) #95s 15.9 bits of security, 88s - 39

        try:
            solutions = gb.variety()
        except ValueError: #non-zero ideal
            continue
            
        for solution in solutions:
            print(gb)
            print(solutions)
            points = [E.lift_x(solution[variablesX[k]]) for k in range(m)]
            
            for v in VectorSpace(GF(2), m):
                if sum(-points[k] if v[k] else points[k] for k in range(m)) == ident:
                    sumA = 0
                    sumB = 0
                    
                    for k in range(m):
                        baseId = factorBase[points[k][0]]
                        sgn = -1 if v[k] else 1

                        if (coord[baseId][0]*P + coord[baseId][1]*Q) == (sgn*points[k]):
                            sumA -= coord[baseId][0]
                            sumB -= coord[baseId][1]
                        else: 
                            sumA += coord[baseId][0]
                            sumB += coord[baseId][1]
                    res = solveECDLP(sumA, sumB, curveOrder)
    
        it += 1
        coord = []
        #dict to map numbers in FF to columns in relationMatrix
        factorBase = {}
        colId = 0
        while colId < fbs:
        #generate random basis
        ## generating an x-coordinate
            a = int(1 + (random.random() * (orderP - 1)))
            b = int(1 + (random.random() * (orderP - 1)))
            candX = (a*P + b*Q)[0]
            if candX not in factorBase.keys() and E.is_x_coord(candX):
                baseBound[mod(colId, m)] *= FBR(X - candX) #add to the m-th base          
                factorBase[candX] = colId
                colId += 1
    
    #bound all x_1,x_2,...,x_m to be in factorBase and prepare a binding variable u_1,...,u_m-2
        for k in range(0,m):
            generators[k] = (baseBound[k](variablesX[k]))
  #  print("Solution: {}, return after {} iterations. Valid result: {}. Took {} seconds."\
   #       .format(res, it, res*P == Q,time.time() - startTime))
    return res
    

#### Tests:
bits = 10
E = generateRandomEC(bits=bits, primeOrder=True)
print(E)
sys.stdout.flush()
P, Q = initECDLP(E)

orderP = P.order()
deg = 7
for k in range(2,20):
    if calcFactorBaseSize(orderP, k) <= k*deg:
        m = k
        break

m = 3
print(E)
print("orderP: {}, m = {}.".format(orderP, m))
sys.stdout.flush()

#sumPolySplitKnownDecomp(Q, P, m)
printMaxMemUsage()
print("\n")

sumPolyOrigApproachSplitGeneral2(Q, P, m)

Elliptic Curve defined by y^2 = x^3 + 339*x + 600 over Finite Field of size 1031
Elliptic Curve defined by y^2 = x^3 + 339*x + 600 over Finite Field of size 1031
orderP: 1069, m = 3.
Maximum memory usage during this session was: 239.08203125 MegaBytes.


Base of size: 11 generated. Entering the main cycle.
Ideal (x0 + 180, x1 + 122, x2 + 67) of Multivariate Polynomial Ring in x0, x1, x2 over Finite Field of size 1031
[{x2: 964, x1: 909, x0: 851}]


KeyError: 851

In [142]:
print(N(log(4911016471,2)))

32.1933745151654


In [166]:
E = EllipticCurve(GF(29),[0,0,0,2,1])
sm = construct3rdSumPoly(E)
FBR.<X,Y> = PolynomialRing(GF(29), 2, order='lex') 
print(sm)
print(sm.subs(x1=X, x2=Y,x3=x3))

x1^2*x2^2 - 2*x1^2*x2*x3 - 2*x1*x2^2*x3 + x1^2*x3^2 - 2*x1*x2*x3^2 + x2^2*x3^2 - 4*x1*x2 - 4*x1*x3 - 4*x2*x3 - 4*x1 - 4*x2 - 4*x3 + 4


NameError: name 'x3' is not defined

In [29]:
#({480: 0, 897: 3, 675: 5, 164: 2, 619: 7, 622: 8, 947: 4, 980: 6, 505: 9, 575: 1},
# [x0^2*x1^2 - 2*x0^2*x1*x3 - 2*x0*x1^2*x3 + x0^2*x3^2 - 2*x0*x1*x3^2 + x1^2*x3^2 - 61*x0*x1 - 61*x0*x3 - 61*x1*x3 + 270*x0 + 270*x1 + 270*x3 + 157, x0^4 + 231*x0^3 - 6*x0^2 - 467*x0 + 288, x1^3 - 79*x1^2 - 59*x1 - 269, x2^3 - 430*x2^2 - 476*x2 - 65])

Q.<x2> = PolynomialRing(GF(991))
f=(x2**3 - 430*x2**2 - 476*x2 - 65)
print(f.roots())

[(283, 1)]


In [59]:
a=[1,2]
b=a[2:]
print(len(b),b)

(0, [])
